In [1]:
import pandas as pd

считываем csv файл и делаем ID в столбце индекса

In [66]:
df = pd.read_csv("../../datasets/auto.csv").set_index("ID")

In [67]:
df.head()

,CarNumber,Make_n_model,Refund,Fines,History
ID,,,,,
0,Y163O8161RUS,Ford Focus,2.0,3200.0,NaN
1,E432XX77RUS,Toyota Camry,1.0,6500.0,NaN
2,7184TT36RUS,Ford Focus,1.0,2100.0,NaN
3,X582HE161RUS,Ford Focus,2.0,2000.0,NaN
4,E34877152RUS,Ford Focus,2.0,6100.0,NaN


считаем количество наблюдений

In [68]:
print(df.count())

CarNumber       931
Make_n_model    931
Refund          914
Fines           869
History          82
dtype: int64


отбрасываем дубликаты,из двух равных наблюдений выбираем последнее(используем только 3 столбца) и считаем количество заново

In [69]:
df = df.drop_duplicates(subset=['CarNumber', 'Make_n_model', 'Fines'], keep='last')
print(df.count())

CarNumber       725
Make_n_model    725
Refund          713
Fines           665
History          65
dtype: int64


проверяем сколько значений отсутствует в каждом столбце,удаляем столбцы с более чем 500 пропусков;заменяем все отсутствующие значения в Refund столбце предыдущим значением в этом столбце для этой ячейки;
заменяем все пропущенные значения в Fines столбце средним значением этого столбца (исключите значения NA/NULL при вычислении среднего значения) и проверям сколько значений пропущено в каждом столбце

In [70]:
print(df.isna().sum())

CarNumber         0
Make_n_model      0
Refund           12
Fines            60
History         660
dtype: int64


In [71]:
df = df.dropna(thresh=len(df)-500, axis=1)
print(df.isna().sum())

CarNumber        0
Make_n_model     0
Refund          12
Fines           60
dtype: int64


In [72]:
df['Refund'] = df['Refund'].ffill()
df['Fines'] = df['Fines'].fillna(df['Fines'].mean())
print(df.isna().sum())

CarNumber       0
Make_n_model    0
Refund          0
Fines           0
dtype: int64


извлекли марку и модель машин в новые столбцы(make,model),удалили столбец make_n_model и сохранили в json файл

In [73]:
df[['Make', 'Model']] = df['Make_n_model'].apply(lambda x: pd.Series(str(x).split(' ', 1)) if pd.notna(x) else pd.Series([None, None]))
df = df.drop('Make_n_model', axis=1)
df.reset_index(drop=True).to_json('../../datasets/auto.json', orient='records', indent=2)
print(df.head())

       CarNumber  Refund   Fines    Make  Model
ID                                             
0   Y163O8161RUS     2.0  3200.0    Ford  Focus
1    E432XX77RUS     1.0  6500.0  Toyota  Camry
2    7184TT36RUS     1.0  2100.0    Ford  Focus
3   X582HE161RUS     2.0  2000.0    Ford  Focus
5   92918M178RUS     1.0  5700.0    Ford  Focus


проверка по чек листу

In [74]:
df2 = pd.read_json('../../datasets/auto.json',orient='records')

In [75]:
print(df2.count())

CarNumber    725
Refund       725
Fines        725
Make         725
Model        716
dtype: int64


In [76]:
print(df2['Fines'].mean())
print(df2['Refund'].mean())

8594.586466165412
1.5172413793103448
